In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/swoop/?sortby=post_date%3ADesc&pagesize=100'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'swoop'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/swoop.csv',index=False)

In [3]:
# Sanity check
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
95,swoop,J Loftus,2019-12-23,✅ Trip Verified | Hamilton to Orlando. I’m wr...,Family Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no
96,swoop,Tannis Coleman,2019-12-23,✅ Trip Verified | Winnipeg to Hamilton. Fligh...,Solo Leisure,Economy Class,2,2,3,NaN,NaN,4,2,no
97,swoop,R Lee,2019-12-22,Not Verified | Hamilton to Fort Lauderdale. Tr...,Family Leisure,Economy Class,1,1,1,NaN,1,1,1,no
98,swoop,S Baird,2019-12-21,✅ Trip Verified | Kelowna to Las Vegas. I was...,Solo Leisure,Economy Class,8,4,4,NaN,NaN,4,5,yes
99,swoop,Ashley Baxter,2019-12-20,Not Verified | I was supposed to go to Las Ve...,Couple Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no


In [4]:
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,swoop,Abdullahi Dawud,2020-06-05,✅ Trip Verified | Edmonton to Hamilton. I was...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no
1,swoop,Tiffany Nguyen,2020-05-02,✅ Trip Verified | Abbostford to Hamilton on 1...,Family Leisure,Economy Class,10,5,5,NaN,NaN,5,5,yes
2,swoop,Greg Scott,2020-04-30,Not Verified | Got my flight to Winnipeg. Ori...,Family Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no
3,swoop,Penny Mckelvie,2020-04-21,✅ Trip Verified | Our flight between Edmonton...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
4,swoop,E Smith,2020-03-30,Not Verified | We used to really like flying ...,Couple Leisure,Economy Class,1,4,4,NaN,NaN,1,4,no
